This notebook presents an application semi-physical model of the L1 laser in its second version, based on PyTorch.
In particular, in this notebook the optimization process of the control parameters is solved using such a model. 

Author: Francesco Capuano, 2022 S17 summer intern @ ELI-beamlines, Prague


# Motivation

The goal of this project is to maximise second-harmonic efficiency. However, since this metric is also very much related to the shortest possible pulse shape, we started with developing a strategy to optimise a predefinite set of control parameters so as to minimise the difference between the obtained pulse shape (in the temporal domain) and a target one (which, by default, is the shortest one typically). 

However, since data are really expensive to empirically collect we resorted to model the underlying dynamics of the whole system, also considering that (even if not exhaustive) there is a significant amount of know-how concerned with the considered dynamics available.

After this model is obtained, it is possible to use it to obtain the desired control parameters.

In [1]:
import torch
# these import are necessary to import modules from directories one level back in the folder structure
import sys
sys.path.append("../..")
from utils.se import get_project_root
from algorithms.L1_BayesianOptimisation import extract_data
import matplotlib.pyplot as plt
from scipy.optimize import Bounds
import numpy as np

frequency, field = extract_data()

c:\Users\Ext_Capuano\Anaconda3\envs\elienv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


The preprocessing steps do not depend on the control parameters, therefore they can take place even in numpy

In [2]:
# preprocessing steps
from utils.physics import *
# preprocessing
cutoff = np.array((289.95, 291.91)) * 1e12
# cutting off the signal
frequency_clean, field_clean = cutoff_signal(frequency_cutoff = cutoff, frequency = frequency * 1e12,
                                             signal = field)
# augmenting the signal
frequency_clean_aug, field_clean_aug = equidistant_points(frequency = frequency_clean,
                                                          signal = field_clean,
                                                          num_points = int(3e3)) # n_points defaults to 5e3
# retrieving central carrier
central_carrier = central_frequency(frequency = frequency_clean_aug, signal = field_clean_aug)

However, to be used in the Computational Laser model, their tensir version is required

In [3]:
from utils.LaserModel_torch import ComputationalLaser as CL

intensity = torch.from_numpy(field ** 2)
frequency, field = torch.from_numpy(frequency_clean_aug), torch.from_numpy(field_clean_aug)
compressor_params = -1 * torch.tensor([267.422 * 1e-24, -2.384 * 1e-36, 9.54893 * 1e-50], dtype = torch.double)

laser = CL(frequency = frequency * 1e-12, field = field, compressor_params = compressor_params)
target_time, target_profile = laser.transform_limited()

Pytorch offers various optimizers which are normally considered to be very well suited in NN. However, with a slight tweak and flexibility of reasoning, they can be applied to this very problem as well, as long as this very problem is actually formulated as one of those Pytorch optimizers aer meant to solve. 

In [4]:
class LaserOptimization(torch.nn.Module): 
    """Custom Pytorch model for gradient based optimization.
    """
    def __init__(self):
        
        super().__init__()
        bounds_control = Bounds(
                    # GDD         # TOD          # FOD
            lb = (2.3522e-22, -1.003635e-34, 4.774465e-50),
            ub = (2.99624e-22, 9.55955e-35, 1.4323395e-49)
        )

        bounds_matrix = np.vstack((bounds_control.lb, bounds_control.ub)).T
        # initialize weights with random numbers
        control = torch.distributions.Uniform(
            low = torch.from_numpy(bounds_matrix[:, 0]), 
            high = torch.from_numpy(bounds_matrix[:, 1])
        ).sample()

        # make weights torch parameters
        self.control = torch.nn.Parameter(control).cuda()    
        
    def objective_function(self, control:torch.tensor) -> float:
        """
        Implements the function to be minimised. In this case such a function will be the L1 norm corrected 
        with a log barrier to maintain the parameters into the feasible region.
        
        Args: 
            
        """
    
    def training_loop(model, optimizer, n=1000):
        "Training loop for torch model."
        losses = []
        for i in range(n):
            preds = model(x)
            loss = F.mse_loss(preds, y).sqrt()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss)  
        return losses

In [5]:
lo = LaserOptimization()

RuntimeError: No CUDA GPUs are available